In [15]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
from skimage.color import rgb2lab

### Core Conversions

In [16]:
def load_rgb(path: str) -> np.ndarray:
    """RGB float32 in [0,1], shape (H,W,3)."""
    img = Image.open(path).convert("RGB")
    return np.asarray(img, dtype=np.float32) / 255.0

def luminance_Lstar(rgb_srgb: np.ndarray) -> np.ndarray:
    """CIELAB L* in [0,100], shape (H,W)."""
    lab = rgb2lab(np.clip(rgb_srgb, 0.0, 1.0))
    return lab[..., 0].astype(np.float32)

In [17]:
def mild_gamma(L: np.ndarray, gamma: float = 1.0) -> np.ndarray:
    """Apply mild gamma to L* (0..100) or [0..1] luminance."""
    if gamma == 1.0:
        return L.astype(np.float32)

    L = np.asarray(L, dtype=np.float32)
    # Treat as L* if values look like 0..100
    if np.nanmax(L) > 1.5:
        Ln = np.clip(L / 100.0, 0.0, 1.0) ** gamma
        return (Ln * 100.0).astype(np.float32)
    else:
        return (np.clip(L, 0.0, 1.0) ** gamma).astype(np.float32)

In [18]:
def save_Lstar_png(L: np.ndarray, out_path: str):
    """
    Save L* as 8-bit PNG for quick viewing.
    L expected in [0..100].
    """
    out_path = str(out_path)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    L8 = np.clip((L / 100.0) * 255.0, 0, 255).astype(np.uint8)
    Image.fromarray(L8, mode="L").save(out_path)

def save_npy(arr: np.ndarray, out_path: str):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    np.save(out_path, arr)

In [19]:
IMG_EXTS = {".png", ".jpg", ".jpeg", ".webp", ".bmp", ".tif", ".tiff"}

def iter_images(folder: Path):
    for p in sorted(folder.rglob("*")):
        if p.is_file() and p.suffix.lower() in IMG_EXTS:
            yield p

def step0_prepare_luminance(
    data_root: str = "data",
    gamma: float = 1.0,
    save_preview_png: bool = True,
    save_npy_arrays: bool = False,
    out_root: str = "artifacts/light_step0"
) -> pd.DataFrame:
    """
    Creates luminance artifacts for real/ and generated/ inside data_root.
    Returns a dataframe manifest with paths + basic stats.
    """
    data_root = Path(data_root)
    out_root = Path(out_root)

    rows = []
    for label in ["real", "generated"]:
        in_dir = data_root / label
        if not in_dir.exists():
            print(f"[WARN] Missing folder: {in_dir} (skipping)")
            continue

        for img_path in iter_images(in_dir):
            try:
                rgb = load_rgb(str(img_path))
                L = luminance_Lstar(rgb)          # 0..100
                L = mild_gamma(L, gamma=gamma)

                rel = img_path.relative_to(data_root)  # e.g., generated/foo.png
                stem = img_path.stem
                subdir = rel.parent  # real/ or generated/ plus any nested

                # Output paths
                preview_path = out_root / "Lstar_png" / subdir / f"{stem}_Lstar.png"
                npy_path = out_root / "Lstar_npy" / subdir / f"{stem}_Lstar.npy"

                if save_preview_png:
                    save_Lstar_png(L, str(preview_path))
                if save_npy_arrays:
                    save_npy(L, str(npy_path))

                rows.append({
                    "split": label,
                    "image_path": str(img_path),
                    "height": int(rgb.shape[0]),
                    "width": int(rgb.shape[1]),
                    "L_min": float(np.nanmin(L)),
                    "L_mean": float(np.nanmean(L)),
                    "L_std": float(np.nanstd(L)),
                    "L_max": float(np.nanmax(L)),
                    "Lstar_png": str(preview_path) if save_preview_png else None,
                    "Lstar_npy": str(npy_path) if save_npy_arrays else None,
                })

            except Exception as e:
                rows.append({
                    "split": label,
                    "image_path": str(img_path),
                    "error": str(e),
                })

    df = pd.DataFrame(rows)
    return df

In [20]:
df0 = step0_prepare_luminance(
    data_root="data",
    gamma=1.0,                 # keep 1.0 unless you see crushed blacks
    save_preview_png=True,
    save_npy_arrays=False,     # switch to True if you want lossless arrays saved
    out_root="artifacts/light_step0"
)

df0.head(), df0["split"].value_counts()

(  split                                         image_path  height  width  \
 0  real  data\real\images\10pm_feeding_around_the_clock...     749    750   
 1  real                         data\real\images\11pm.webp     642    500   
 2  real                    data\real\images\12_4_7_10.webp     749    750   
 3  real             data\real\images\around_the_clock.webp     799    750   
 4  real    data\real\images\around_the_clock_alizarin.webp     704   1000   
 
        L_min     L_mean      L_std      L_max  \
 0   5.593327  52.110363  21.899624  94.124046   
 1   1.337456  54.662834  25.377407  93.048141   
 2   0.595446  40.513443  30.257240  99.507088   
 3  12.022461  58.663769  25.534916  94.476654   
 4   0.349852  46.906425  32.492565  93.133461   
 
                                            Lstar_png Lstar_npy  
 0  artifacts\light_step0\Lstar_png\real\images\10...      None  
 1  artifacts\light_step0\Lstar_png\real\images\11...      None  
 2  artifacts\light_step0\Lsta

### Multi-scale Retinex on L*

In [21]:
import numpy as np
from scipy.ndimage import gaussian_filter

def multiscale_retinex(L, sigmas=(15, 80, 250), weights=None, eps=1e-6):
    """
    Multi-scale Retinex for a luminance/lightness map.
    Input:
      L: 2D array (H,W). Can be L* in [0,100] or luminance in [0,1].
    Output:
      R: 2D array (H,W), retinex response (float32), unnormalized.
    """
    L = np.asarray(L, dtype=np.float32)
    # If L* [0..100], convert to [0..1] for numerics
    if np.nanmax(L) > 1.5:
        L = np.clip(L / 100.0, 0.0, 1.0)
    else:
        L = np.clip(L, 0.0, 1.0)

    if weights is None:
        weights = np.ones(len(sigmas), dtype=np.float32) / len(sigmas)
    else:
        weights = np.asarray(weights, dtype=np.float32)
        weights = weights / (weights.sum() + 1e-12)

    logL = np.log(L + eps)
    R = np.zeros_like(L, dtype=np.float32)

    for w, s in zip(weights, sigmas):
        surround = gaussian_filter(L, sigma=s, mode="reflect")
        R += w * (logL - np.log(surround + eps))

    return R


### Robust normalization - using percentiles

In [22]:
def robust_normalize(R, low_q=1.0, high_q=99.0, out_range=(0.0, 1.0)):
    """
    Normalize R using percentile clipping -> linear rescale.
    """
    R = np.asarray(R, dtype=np.float32)
    lo = np.nanpercentile(R, low_q)
    hi = np.nanpercentile(R, high_q)
    if hi - lo < 1e-8:
        return np.full_like(R, (out_range[0] + out_range[1]) / 2, dtype=np.float32)

    Rc = np.clip(R, lo, hi)
    Rn = (Rc - lo) / (hi - lo)
    a, b = out_range
    return (a + (b - a) * Rn).astype(np.float32)


### Apply to dataset

In [23]:
def save_gray01_png(M01, out_path):
    """Save a [0,1] map as 8-bit grayscale PNG."""
    os.makedirs(os.path.dirname(str(out_path)), exist_ok=True)
    im = (np.clip(M01, 0.0, 1.0) * 255.0).astype(np.uint8)
    Image.fromarray(im, mode="L").save(str(out_path))

def step1_compute_retinex(
    df0: pd.DataFrame,
    out_root="artifacts/light_step1",
    sigmas=(15, 80, 250),
    weights=None,
    norm_q=(1.0, 99.0),
    eps=1e-6
) -> pd.DataFrame:
    """
    For each image in df0, compute Retinex relative brightness map from L*,
    store preview PNG, and return updated df1.
    """
    out_root = Path(out_root)
    rows = []

    for _, row in df0.iterrows():
        if "error" in row and pd.notna(row["error"]):
            rows.append({**row})
            continue

        img_path = Path(row["image_path"])
        split = row["split"]
        stem = img_path.stem

        # Recompute L* from original RGB (keeps df0 optional)
        rgb = load_rgb(str(img_path))
        L = luminance_Lstar(rgb)  # 0..100

        # Retinex
        R = multiscale_retinex(L, sigmas=sigmas, weights=weights, eps=eps)
        R01 = robust_normalize(R, low_q=norm_q[0], high_q=norm_q[1], out_range=(0.0, 1.0))

        # Save preview
        preview_path = out_root / "retinex_png" / split / f"{stem}_retinex.png"
        save_gray01_png(R01, preview_path)

        # Lightness field stats (useful later)
        rows.append({
            **row,
            "retinex_path": str(preview_path),
            "retinex_min": float(np.nanmin(R01)),
            "retinex_mean": float(np.nanmean(R01)),
            "retinex_std": float(np.nanstd(R01)),
            "retinex_max": float(np.nanmax(R01)),
            "retinex_sigmas": str(tuple(sigmas)),
            "retinex_norm_q": str(tuple(norm_q)),
        })

    return pd.DataFrame(rows)

In [24]:
df1 = step1_compute_retinex(
    df0,
    out_root="artifacts/light_step1",
    sigmas=(15, 80, 250),
    norm_q=(1, 99)
)

df1.head()


,split,image_path,height,width,L_min,L_mean,L_std,L_max,Lstar_png,Lstar_npy,retinex_path,retinex_min,retinex_mean,retinex_std,retinex_max,retinex_sigmas,retinex_norm_q
0,real,data\real\images\10pm_feeding_around_the_clock...,749,750,5.593327,52.110363,21.899624,94.124046,artifacts\light_step0\Lstar_png\real\images\10...,None,artifacts\light_step1\retinex_png\real\10pm_fe...,0.0,0.624693,0.222949,1.0,"(15, 80, 250)","(1, 99)"
1,real,data\real\images\11pm.webp,642,500,1.337456,54.662834,25.377407,93.048141,artifacts\light_step0\Lstar_png\real\images\11...,None,artifacts\light_step1\retinex_png\real\11pm_re...,0.0,0.726775,0.194081,1.0,"(15, 80, 250)","(1, 99)"
2,real,data\real\images\12_4_7_10.webp,749,750,0.595446,40.513443,30.257240,99.507088,artifacts\light_step0\Lstar_png\real\images\12...,None,artifacts\light_step1\retinex_png\real\12_4_7_...,0.0,0.718103,0.214318,1.0,"(15, 80, 250)","(1, 99)"
3,real,data\real\images\around_the_clock.webp,799,750,12.022461,58.663769,25.534916,94.476654,artifacts\light_step0\Lstar_png\real\images\ar...,None,artifacts\light_step1\retinex_png\real\around_...,0.0,0.655815,0.245091,1.0,"(15, 80, 250)","(1, 99)"
4,real,data\real\images\around_the_clock_alizarin.webp,704,1000,0.349852,46.906425,32.492565,93.133461,artifacts\light_step0\Lstar_png\real\images\ar...,None,artifacts\light_step1\retinex_png\real\around_...,0.0,0.701177,0.241964,1.0,"(15, 80, 250)","(1, 99)"


In [25]:
df1.groupby("split")[["retinex_mean","retinex_std"]].describe()

retinex_mean                                                    \
                 count      mean       std       min       25%       50%   
split                                                                      
generated         65.0  0.601382  0.116669  0.302159  0.524915  0.609745   
real              65.0  0.698055  0.059071  0.539986  0.672552  0.701873   

                              retinex_std                                \
                75%       max       count      mean       std       min   
split                                                                     
generated  0.668818  0.887587        65.0  0.226496  0.035644  0.142782   
real       0.730392  0.821215        65.0  0.222635  0.029750  0.158227   

                                                   
                25%       50%       75%       max  
split                                              
generated  0.203481  0.224119  0.253416  0.302531  
real       0.196794  0.221347  0.246014  0.287976

### Analytical intrinsic decomposition proxy (Retinex-consistent)

In [26]:
from scipy.ndimage import gaussian_filter

def intrinsic_decompose_proxy(L, sigmas=(30, 120), weights=None, eps=1e-6):
    """
    Intrinsic decomposition proxy:
      L -> Reflectance (R) + Shading (S)
    Using a multi-scale smooth illumination field in log domain.

    Inputs:
      L: 2D array, can be L* [0..100] or luminance [0..1]
    Returns:
      R01: reflectance-like map in [0,1] (normalized)
      S01: shading/illumination map in [0,1] (normalized)
      logR: raw log-reflectance (unnormalized, float)
      logS: raw log-shading (unnormalized, float)
    """

    L = np.asarray(L, dtype=np.float32)

    # Convert L* -> [0,1] luminance-ish for stable math
    if np.nanmax(L) > 1.5:
        L01 = np.clip(L / 100.0, 0.0, 1.0)
    else:
        L01 = np.clip(L, 0.0, 1.0)

    if weights is None:
        weights = np.ones(len(sigmas), dtype=np.float32) / len(sigmas)
    else:
        weights = np.asarray(weights, dtype=np.float32)
        weights = weights / (weights.sum() + 1e-12)

    logL = np.log(L01 + eps)

    # Multi-scale estimate of log-shading (smooth field)
    logS = np.zeros_like(logL, dtype=np.float32)
    for w, s in zip(weights, sigmas):
        logS += w * gaussian_filter(logL, sigma=s, mode="reflect")

    # Log reflectance = logL - logS
    logR = logL - logS

    # Normalize to [0,1] robustly for viewing/metrics
    R01 = robust_normalize(logR, low_q=1, high_q=99, out_range=(0, 1))
    S01 = robust_normalize(logS,  low_q=1, high_q=99, out_range=(0, 1))

    return R01, S01, logR, logS

In [27]:
def save_gray01_png(M01, out_path):
    os.makedirs(os.path.dirname(str(out_path)), exist_ok=True)
    im = (np.clip(M01, 0.0, 1.0) * 255.0).astype(np.uint8)
    Image.fromarray(im, mode="L").save(str(out_path))

def step2_run_intrinsic_proxy(
    df0: pd.DataFrame,
    out_root="artifacts/light_step2",
    sigmas=(30, 120)
) -> pd.DataFrame:

    out_root = Path(out_root)
    rows = []

    for _, row in df0.iterrows():
        if "error" in row and pd.notna(row["error"]):
            rows.append({**row})
            continue

        img_path = Path(row["image_path"])
        split = row["split"]
        stem  = img_path.stem

        rgb = load_rgb(str(img_path))
        L   = luminance_Lstar(rgb)  # 0..100

        R01, S01, logR, logS = intrinsic_decompose_proxy(L, sigmas=sigmas)

        # Save previews
        r_path = out_root / "reflectance_png" / split / f"{stem}_R.png"
        s_path = out_root / "shading_png"     / split / f"{stem}_S.png"
        save_gray01_png(R01, r_path)
        save_gray01_png(S01, s_path)

        rows.append({
            **row,
            "R_path": str(r_path),
            "S_path": str(s_path),
            "R_mean": float(np.mean(R01)),
            "R_std":  float(np.std(R01)),
            "S_mean": float(np.mean(S01)),
            "S_std":  float(np.std(S01)),
            "iid_sigmas": str(tuple(sigmas)),
        })

    return pd.DataFrame(rows)

In [28]:
df2 = step2_run_intrinsic_proxy(
    df0,
    out_root="artifacts/light_step2",
    sigmas=(30, 120)
)

df2.head()

,split,image_path,height,width,L_min,L_mean,L_std,L_max,Lstar_png,Lstar_npy,R_path,S_path,R_mean,R_std,S_mean,S_std,iid_sigmas
0,real,data\real\images\10pm_feeding_around_the_clock...,749,750,5.593327,52.110363,21.899624,94.124046,artifacts\light_step0\Lstar_png\real\images\10...,None,artifacts\light_step2\reflectance_png\real\10p...,artifacts\light_step2\shading_png\real\10pm_fe...,0.524036,0.198521,0.610980,0.248655,"(30, 120)"
1,real,data\real\images\11pm.webp,642,500,1.337456,54.662834,25.377407,93.048141,artifacts\light_step0\Lstar_png\real\images\11...,None,artifacts\light_step2\reflectance_png\real\11p...,artifacts\light_step2\shading_png\real\11pm_S.png,0.637681,0.182548,0.596490,0.260964,"(30, 120)"
2,real,data\real\images\12_4_7_10.webp,749,750,0.595446,40.513443,30.257240,99.507088,artifacts\light_step0\Lstar_png\real\images\12...,None,artifacts\light_step2\reflectance_png\real\12_...,artifacts\light_step2\shading_png\real\12_4_7_...,0.596369,0.209352,0.643771,0.197648,"(30, 120)"
3,real,data\real\images\around_the_clock.webp,799,750,12.022461,58.663769,25.534916,94.476654,artifacts\light_step0\Lstar_png\real\images\ar...,None,artifacts\light_step2\reflectance_png\real\aro...,artifacts\light_step2\shading_png\real\around_...,0.564847,0.225639,0.563593,0.257092,"(30, 120)"
4,real,data\real\images\around_the_clock_alizarin.webp,704,1000,0.349852,46.906425,32.492565,93.133461,artifacts\light_step0\Lstar_png\real\images\ar...,None,artifacts\light_step2\reflectance_png\real\aro...,artifacts\light_step2\shading_png\real\around_...,0.545890,0.215944,0.625835,0.265996,"(30, 120)"


### Core Metrics on Shading Map

In [29]:
from scipy.ndimage import sobel

def shading_metrics(S01: np.ndarray, mag_eps=1e-6, high_q=95):
    """
    Compute gradient + ordinal coherence metrics on shading map S in [0,1].
    Returns a dict of scalar metrics.
    """
    S = np.asarray(S01, dtype=np.float32)
    S = np.clip(S, 0.0, 1.0)

    # --- Gradients (Sobel) ---
    gx = sobel(S, axis=1, mode="reflect") / 8.0
    gy = sobel(S, axis=0, mode="reflect") / 8.0
    mag = np.sqrt(gx * gx + gy * gy) + mag_eps
    theta = np.arctan2(gy, gx)

    # --- Smoothness / disruption ---
    tv_mean = float(np.mean(mag))                  # Total variation (mean gradient magnitude)
    tv_median = float(np.median(mag))
    p95 = float(np.percentile(mag, high_q))
    high_ratio = float(np.mean(mag >= p95))        # by definition ~5%, but useful across fixed-q pipeline
    # Better: fixed absolute threshold relative to distribution
    thr = float(np.percentile(mag, 90))
    disrupt_ratio = float(np.mean(mag >= thr))     # % of "strong edges" in shading (breaks)

    # --- Directional coherence (weighted circular mean resultant length) ---
    # Weight by magnitude, but ignore near-flat regions (below median)
    w = mag.copy()
    w[mag < np.median(mag)] = 0.0
    W = float(np.sum(w)) + 1e-12

    vx = float(np.sum(w * np.cos(theta)) / W)
    vy = float(np.sum(w * np.sin(theta)) / W)
    dir_coherence = float(np.sqrt(vx * vx + vy * vy))  # 0..1

    # --- Ordinal relations (4-neighborhood) ---
    # Compare to right and down neighbors to avoid double counting
    right = np.roll(S, shift=-1, axis=1)
    down  = np.roll(S, shift=-1, axis=0)

    # exclude last row/col roll artifacts by masking
    mask_r = np.ones_like(S, dtype=bool); mask_r[:, -1] = False
    mask_d = np.ones_like(S, dtype=bool); mask_d[-1, :] = False

    # ordinal comparisons
    lighter_r = (right > S) & mask_r
    darker_r  = (right < S) & mask_r
    lighter_d = (down  > S) & mask_d
    darker_d  = (down  < S) & mask_d

    total_comp = float(np.sum(mask_r) + np.sum(mask_d)) + 1e-12
    ordinal_lighter = float((np.sum(lighter_r) + np.sum(lighter_d)) / total_comp)
    ordinal_darker  = float((np.sum(darker_r)  + np.sum(darker_d))  / total_comp)
    ordinal_equal   = float(1.0 - ordinal_lighter - ordinal_darker)

    # A simple “ordinal instability” proxy:
    # lots of equal is smooth/flat; lots of flips (both lighter & darker high) suggests irregularity
    ordinal_instability = float(min(ordinal_lighter, ordinal_darker) * 2.0)  # 0..1

    return {
        "tv_mean": tv_mean,
        "tv_median": tv_median,
        "mag_p95": p95,
        "disrupt_ratio_p90": disrupt_ratio,
        "dir_coherence": dir_coherence,
        "ordinal_lighter": ordinal_lighter,
        "ordinal_darker": ordinal_darker,
        "ordinal_equal": ordinal_equal,
        "ordinal_instability": ordinal_instability,
    }

In [30]:
from PIL import Image
import pandas as pd

def load_gray01(path: str) -> np.ndarray:
    """Load a grayscale PNG and return float32 in [0,1]."""
    im = Image.open(path).convert("L")
    arr = np.asarray(im, dtype=np.float32) / 255.0
    return arr

def step3_compute_shading_coherence(df2: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for _, row in df2.iterrows():
        if "error" in row and pd.notna(row["error"]):
            rows.append({**row})
            continue

        S_path = row.get("S_path", None)
        if not S_path or (isinstance(S_path, float) and np.isnan(S_path)):
            rows.append({**row, "error": "Missing S_path"})
            continue

        try:
            S = load_gray01(S_path)
            m = shading_metrics(S)

            rows.append({**row, **{
                "S_tv_mean": m["tv_mean"],
                "S_tv_median": m["tv_median"],
                "S_mag_p95": m["mag_p95"],
                "S_disrupt_ratio_p90": m["disrupt_ratio_p90"],
                "S_dir_coherence": m["dir_coherence"],
                "S_ord_lighter": m["ordinal_lighter"],
                "S_ord_darker": m["ordinal_darker"],
                "S_ord_equal": m["ordinal_equal"],
                "S_ord_instability": m["ordinal_instability"],
            }})
        except Exception as e:
            rows.append({**row, "error": str(e)})

    return pd.DataFrame(rows)

In [31]:
df3 = step3_compute_shading_coherence(df2)
df3.groupby("split")[["S_tv_mean","S_dir_coherence","S_disrupt_ratio_p90","S_ord_instability"]].describe()

S_tv_mean                                                    \
              count      mean       std       min       25%       50%   
split                                                                   
generated      65.0  0.003752  0.001016  0.001928  0.003042  0.003638   
real           65.0  0.003222  0.000937  0.000839  0.002688  0.003062   

                              S_dir_coherence            ...  \
                75%       max           count      mean  ...   
split                                                    ...   
generated  0.004257  0.006236            65.0  0.197400  ...   
real       0.003808  0.005661            65.0  0.054487  ...   

          S_disrupt_ratio_p90           S_ord_instability                      \
                          75%       max             count      mean       std   
split                                                                           
generated            0.107395  0.141403              65.0  0.412536  0.135020   
real                 0.110280  0.153545              65.0  0.399516  0.107797   

                                                             
                min       25%       50%       75%       max  
split                                                        
generated  0.065921  0.311544  0.407538  0.517590  0.672552  
real       0.125565  0.321336  0.398177  0.474133  0.625036  

[2 rows x 32 columns]

### Shadow

### Shadow mask + Penumbra Metrics + Cast/Attached Proxy

In [33]:
import numpy as np
from PIL import Image
from scipy.ndimage import gaussian_filter, distance_transform_edt, sobel
from skimage.feature import canny
from skimage.morphology import remove_small_objects, binary_opening, disk
from skimage.measure import label, regionprops

def load_gray01(path: str) -> np.ndarray:
    im = Image.open(path).convert("L")
    return (np.asarray(im, dtype=np.float32) / 255.0)

def robust_percentile_threshold(x, q):
    return float(np.percentile(x[np.isfinite(x)], q))

def compute_shadow_mask(S01: np.ndarray, q_shadow=20, smooth_sigma=1.0, min_area_px=200):
    """
    Shadow regions = low values in shading map.
    q_shadow: percentile threshold (lower -> stricter shadow)
    """
    S = np.clip(np.asarray(S01, dtype=np.float32), 0.0, 1.0)
    S_s = gaussian_filter(S, sigma=smooth_sigma, mode="reflect") if smooth_sigma > 0 else S

    thr = robust_percentile_threshold(S_s, q_shadow)   # e.g., 20th percentile
    shadow = (S_s <= thr)

    # cleanup
    shadow = binary_opening(shadow, footprint=disk(1))
    shadow = remove_small_objects(shadow, min_size=min_area_px)

    return shadow, thr, S_s

def penumbra_softness(S_s: np.ndarray, shadow_mask: np.ndarray,
                      band_px=25, eps=1e-6):
    """
    Estimate softness using shading gradient magnitude in a boundary band.
    Soft edge => low gradient at boundary (spread out)
    Hard edge => high gradient at boundary (sharp)
    """
    S = np.asarray(S_s, dtype=np.float32)

    gx = sobel(S, axis=1, mode="reflect") / 8.0
    gy = sobel(S, axis=0, mode="reflect") / 8.0
    mag = np.sqrt(gx*gx + gy*gy) + eps

    # boundary band around shadow edge
    dist_in = distance_transform_edt(shadow_mask)
    dist_out = distance_transform_edt(~shadow_mask)
    boundary_band = (dist_in <= band_px) & (dist_out <= band_px) & (shadow_mask | ~shadow_mask)

    # focus near the actual boundary (thin-ish band)
    boundary = (dist_in <= 2) & (dist_out <= 2)

    # metrics
    mean_grad_boundary = float(np.mean(mag[boundary])) if np.any(boundary) else np.nan
    mean_grad_band = float(np.mean(mag[boundary_band])) if np.any(boundary_band) else np.nan

    # softness score: inverse of boundary sharpness (higher = softer)
    softness = float(1.0 / (mean_grad_boundary + eps)) if np.isfinite(mean_grad_boundary) else np.nan

    return {
        "shadow_edge_grad_mean": mean_grad_boundary,
        "shadow_band_grad_mean": mean_grad_band,
        "penumbra_softness": softness
    }, mag, boundary

def cast_vs_attached_proxy(R01: np.ndarray, shadow_boundary: np.ndarray,
                           canny_sigma=1.5, overlap_tol=1):
    """
    Proxy:
    - Compute reflectance edges from R
    - If shadow boundary overlaps reflectance edges a lot => attached-ish
    - If overlap is low => cast-ish
    """
    R = np.clip(np.asarray(R01, dtype=np.float32), 0.0, 1.0)

    # Canny edges on reflectance
    edges_R = canny(R, sigma=canny_sigma)

    # allow small tolerance: dilate edges a bit via distance
    dist_edges = distance_transform_edt(~edges_R)
    edges_tol = dist_edges <= overlap_tol

    boundary = shadow_boundary.astype(bool)
    if np.sum(boundary) < 10:
        return {"shadow_edge_overlap": np.nan, "castness": np.nan}, edges_R

    overlap = float(np.mean(edges_tol[boundary]))  # fraction of boundary pixels near reflectance edges
    castness = float(1.0 - overlap)               # higher => more “cast-like”

    return {
        "shadow_edge_overlap": overlap,
        "castness": castness
    }, edges_R

def shadow_region_stats(S_s: np.ndarray, shadow_mask: np.ndarray):
    S = np.asarray(S_s, dtype=np.float32)
    sh = shadow_mask.astype(bool)
    if np.sum(sh) < 10:
        return {
            "shadow_area_ratio": 0.0,
            "shadow_mean_S": np.nan,
            "shadow_min_S": np.nan,
            "shadow_components": 0
        }
    comps = label(sh)
    return {
        "shadow_area_ratio": float(np.mean(sh)),
        "shadow_mean_S": float(np.mean(S[sh])),
        "shadow_min_S": float(np.min(S[sh])),
        "shadow_components": int(len(regionprops(comps)))
    }

In [34]:
def step4_shadow_metrics(df2: pd.DataFrame,
                         q_shadow=20,
                         smooth_sigma=1.0,
                         min_area_px=200,
                         band_px=25,
                         canny_sigma=1.5):
    rows = []
    for _, row in df2.iterrows():
        if "error" in row and pd.notna(row["error"]):
            rows.append({**row})
            continue

        S_path = row.get("S_path", None)
        R_path = row.get("R_path", None)
        if not S_path or not R_path:
            rows.append({**row, "error": "Missing S_path or R_path"})
            continue

        try:
            S01 = load_gray01(S_path)
            R01 = load_gray01(R_path)

            shadow_mask, thr, S_s = compute_shadow_mask(
                S01, q_shadow=q_shadow, smooth_sigma=smooth_sigma, min_area_px=min_area_px
            )

            # boundary (2px band around interface)
            dist_in = distance_transform_edt(shadow_mask)
            dist_out = distance_transform_edt(~shadow_mask)
            shadow_boundary = (dist_in <= 2) & (dist_out <= 2)

            pen_metrics, grad_mag, boundary = penumbra_softness(
                S_s, shadow_mask, band_px=band_px
            )

            cast_metrics, edges_R = cast_vs_attached_proxy(
                R01, shadow_boundary, canny_sigma=canny_sigma, overlap_tol=1
            )

            reg_metrics = shadow_region_stats(S_s, shadow_mask)

            rows.append({
                **row,
                "shadow_thr_S": float(thr),
                **reg_metrics,
                **pen_metrics,
                **cast_metrics,
                "shadow_q": q_shadow,
                "shadow_band_px": band_px,
            })

        except Exception as e:
            rows.append({**row, "error": str(e)})

    return pd.DataFrame(rows)

In [35]:
df4 = step4_shadow_metrics(df2, q_shadow=20, smooth_sigma=1.0, min_area_px=200, band_px=25, canny_sigma=1.5)
df4.groupby("split")[["shadow_area_ratio","penumbra_softness","castness","shadow_components"]].describe()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5016\13212555.py:27: FutureWarning: `binary_opening` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.opening` instead.
  shadow = binary_opening(shadow, footprint=disk(1))
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5016\13212555.py:28: FutureWarning: Parameter `min_size` is deprecated since version 0.26.0 and will be removed in 2.0.0 (or later). To avoid this warning, please use the parameter `max_size` instead. For more details, see the documentation of `remove_small_objects`. Note that the new threshold removes objects smaller than **or equal to** its value, while the previous parameter only removed smaller ones.
  shadow = remove_small_objects(shadow, min_size=min_area_px)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5016\13212555.py:27: FutureWarning: `binary_opening` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.opening` instead.
  shadow = b

shadow_area_ratio                                                    \
                      count      mean       std       min       25%       50%   
split                                                                           
generated              65.0  0.199992  0.000077  0.199383  0.199997  0.200001   
real                   65.0  0.199993  0.000056  0.199550  0.200000  0.200000   

                              penumbra_softness              ...  castness  \
                75%       max             count        mean  ...       75%   
split                                                        ...             
generated  0.200001  0.200058              65.0  202.952223  ...  0.967114   
real       0.200000  0.200044              65.0  196.978913  ...  0.932387   

               shadow_components                                               
           max             count      mean       std  min  25%  50%  75%  max  
split                                                                          
generated  1.0              65.0  1.984615  1.138699  1.0  1.0  2.0  2.0  6.0  
real       1.0              65.0  1.984615  1.419642  1.0  1.0  1.0  3.0  7.0  

[2 rows x 32 columns]

### Light-Driven Depth & Spatial Cue Metrics

In [36]:
def depth_gradient_metrics(S01):
    S = gaussian_filter(S01.astype(float), sigma=1.0)

    gx = sobel(S, axis=1) / 8.0
    gy = sobel(S, axis=0) / 8.0
    grad_mag = np.sqrt(gx**2 + gy**2)

    # second derivative (curvature of light field)
    gxx = sobel(gx, axis=1) / 8.0
    gyy = sobel(gy, axis=0) / 8.0
    grad_curv = np.sqrt(gxx**2 + gyy**2)

    return {
        "light_depth_gradient_mean": float(np.mean(grad_mag)),
        "light_depth_gradient_std": float(np.std(grad_mag)),
        "light_gradient_curvature_mean": float(np.mean(grad_curv))
    }, grad_mag

### Brightness Leaps (Spatial Jumps)

In [37]:
def brightness_leap_metrics(grad_mag, p=95):
    thresh = np.percentile(grad_mag, p)
    leaps = grad_mag >= thresh

    return {
        "brightness_leap_ratio": float(np.mean(leaps)),
        "brightness_leap_strength": float(np.mean(grad_mag[leaps])) if np.any(leaps) else 0.0
    }

### Light-Based Figure–Ground Emphasis

In [38]:
def figure_ground_light_metrics(S01, q=20):
    hi = np.percentile(S01, 100-q)
    lo = np.percentile(S01, q)

    fg = S01 >= hi
    bg = S01 <= lo

    if fg.sum() < 10 or bg.sum() < 10:
        return {
            "fg_bg_light_contrast": np.nan,
            "fg_light_mean": np.nan,
            "bg_light_mean": np.nan
        }

    return {
        "fg_bg_light_contrast": float(np.mean(S01[fg]) - np.mean(S01[bg])),
        "fg_light_mean": float(np.mean(S01[fg])),
        "bg_light_mean": float(np.mean(S01[bg]))
    }

In [39]:
def step6_light_depth_metrics(df):
    rows = []
    for _, row in df.iterrows():
        if pd.notna(row.get("error")):
            rows.append({**row})
            continue

        S = load_gray01(row["S_path"])

        depth_m, grad_mag = depth_gradient_metrics(S)
        leap_m = brightness_leap_metrics(grad_mag, p=95)
        fg_m = figure_ground_light_metrics(S, q=20)

        rows.append({
            **row,
            **depth_m,
            **leap_m,
            **fg_m
        })

    return pd.DataFrame(rows)

In [40]:
df6 = step6_light_depth_metrics(df4)
df6.groupby("split")[[
    "light_depth_gradient_mean",
    "light_gradient_curvature_mean",
    "brightness_leap_ratio",
    "fg_bg_light_contrast"
]].describe()


light_depth_gradient_mean                                          \
                              count      mean       std       min       25%   
split                                                                         
generated                      65.0  0.003728  0.001016  0.001898  0.003016   
real                           65.0  0.003198  0.000935  0.000829  0.002666   

                                        light_gradient_curvature_mean  \
                50%       75%       max                         count   
split                                                                   
generated  0.003611  0.004231  0.006211                          65.0   
real       0.003033  0.003784  0.005638                          65.0   

                     ... brightness_leap_ratio           fg_bg_light_contrast  \
               mean  ...                   75%       max                count   
split                ...                                                        
generated  0.000184  ...              0.050003  0.050007                 65.0   
real       0.000169  ...              0.050000  0.050004                 65.0   

                                                                       \
               mean       std       min       25%       50%       75%   
split                                                                   
generated  0.699275  0.101500  0.420354  0.643501  0.710804  0.771064   
real       0.703694  0.078913  0.469387  0.671220  0.715292  0.751488   

                     
                max  
split                
generated  0.879148  
real       0.877369  

[2 rows x 32 columns]

### Attentional Emphasis (Light-Driven)

How unevenly illumination distributes perceptual emphasis across the image. Not where people look, but: whether light concentrates attention or distributes it evenly

### Local luminance contrast (DoG-style, but simple)

In [41]:
def luminance_contrast_map(S01, sigma_local=3, sigma_global=15):
    local = gaussian_filter(S01, sigma=sigma_local)
    global_ = gaussian_filter(S01, sigma=sigma_global)
    return np.abs(local - global_)

### Attentional concentration score

In [42]:
def attentional_concentration(C):
    # normalized entropy-style measure
    C = C[C > 0]
    if len(C) == 0:
        return np.nan
    Cn = C / (C.sum() + 1e-8)
    entropy = -np.sum(Cn * np.log(Cn + 1e-8))
    return float(entropy)

### Highlight / shadow emphasis balance

In [43]:
def highlight_shadow_attention(S01, C, q=20):
    hi = np.percentile(S01, 100-q)
    lo = np.percentile(S01, q)

    highlight_attention = float(np.mean(C[S01 >= hi]))
    shadow_attention = float(np.mean(C[S01 <= lo]))

    return {
        "highlight_attention": highlight_attention,
        "shadow_attention": shadow_attention,
        "hl_shadow_ratio": highlight_attention / (shadow_attention + 1e-6)
    }

Aggregate

In [44]:
def step7_attentional_metrics(df):
    rows = []
    for _, row in df.iterrows():
        if pd.notna(row.get("error")):
            rows.append({**row})
            continue

        S = load_gray01(row["S_path"])

        C = luminance_contrast_map(S)
        entropy = attentional_concentration(C)
        hl = highlight_shadow_attention(S, C)

        rows.append({
            **row,
            "attention_entropy": entropy,
            **hl
        })

    return pd.DataFrame(rows)

In [45]:
df7 = step7_attentional_metrics(df6)
df7.groupby("split")[[
    "attention_entropy",
    "highlight_attention",
    "shadow_attention",
    "hl_shadow_ratio"
]].describe()

attention_entropy                                             \
                      count       mean       std        min        25%   
split                                                                    
generated              65.0  12.143452  0.126388  11.581718  12.104311   
real                   65.0  12.740572  0.597506  12.086586  12.385543   

                                           highlight_attention            ...  \
                 50%        75%        max               count      mean  ...   
split                                                                     ...   
generated  12.159509  12.204936  12.325964                65.0  0.010235  ...   
real       12.487764  12.912476  15.034754                65.0  0.006662  ...   

          shadow_attention           hl_shadow_ratio                      \
                       75%       max           count      mean       std   
split                                                                      
generated         0.018916  0.035623            65.0  0.759626  0.624100   
real              0.014672  0.026608            65.0  0.525123  0.303018   

                                                             
                min       25%       50%       75%       max  
split                                                        
generated  0.077936  0.350422  0.587124  0.958161  3.722833  
real       0.030954  0.300288  0.512078  0.640960  1.370084  

[2 rows x 32 columns]

### Light Organization Metrics

In [57]:
KEYS = ["split", "image_path"]

# ---------- utils ----------
def zscore_global(s: pd.Series) -> pd.Series:
    """Global z-score over all images (real+generated)."""
    s = pd.to_numeric(s, errors="coerce")
    mu = s.mean(skipna=True)
    sd = s.std(skipna=True, ddof=0)
    return (s - mu) / (sd + 1e-12)

def require_columns(df: pd.DataFrame, cols, df_name="df"):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise KeyError(f"{df_name} missing columns: {missing}")
    return True

# ---------- 8.1 Build canonical base table for Step 8 ----------
def build_light_base(df7: pd.DataFrame, df1: pd.DataFrame, df3: pd.DataFrame) -> pd.DataFrame:
    """
    Canonical merge for Step 8.
    - df7 supplies: castness, penumbra_softness, fg_bg_light_contrast, attention_entropy, etc.
    - df1 supplies: retinex_std (relative brightness contrast)
    - df3 supplies: S_tv_mean (illumination smoothness) and related shading coherence metrics
    """
    # Minimal columns needed from each upstream df
    df1_cols = ["split", "image_path", "retinex_std"]
    df3_cols = ["split", "image_path", "S_tv_mean"]  # you can add more later if needed

    require_columns(df1, df1_cols, "df1")
    require_columns(df3, df3_cols, "df3")
    require_columns(df7, ["split", "image_path"], "df7")

    base = (
        df7.merge(df1[df1_cols], on=KEYS, how="left")
           .merge(df3[df3_cols], on=KEYS, how="left")
    )
    return base

def light_base_missing_report(df_light_base: pd.DataFrame) -> pd.Series:
    """Quick missingness report for Step 8 core inputs."""
    need = ["retinex_std", "S_tv_mean", "castness", "penumbra_softness",
            "fg_bg_light_contrast", "attention_entropy"]
    present = [c for c in need if c in df_light_base.columns]
    return df_light_base[present].isna().sum().sort_values(ascending=False)

# ---------- 8.2 Build Minimal LFV (one metric per Arnheim bucket) ----------
def build_light_feature_vector_minimal(df_light_base: pd.DataFrame) -> pd.DataFrame:
    """
    Minimal LFV: 1 metric per Arnheim bucket.
    Produces both raw bucket metrics and z-scored bucket metrics.
    Also outputs LightOrganizationIndex_core and LightOrganizationIndex_all.
    """
    required = [
        "split","image_path",
        "retinex_std",          # Bucket A
        "S_tv_mean",            # Bucket B
        "castness",             # Bucket C
        "penumbra_softness",    # Bucket D
        "fg_bg_light_contrast", # Bucket E
        "attention_entropy"     # Bucket F
    ]
    require_columns(df_light_base, required, "df_light_base")

    df = df_light_base.copy()

    # ---- 1) Raw bucket metrics ----
    # A: Relative brightness contrast
    df["LF_brightness_contrast_raw"] = pd.to_numeric(df["retinex_std"], errors="coerce")

    # B: Illumination smoothness (lower tv => smoother)
    df["LF_illum_smoothness_raw"] = pd.to_numeric(df["S_tv_mean"], errors="coerce")

    # C: Shadow structure (castness proxy)
    df["LF_shadow_structure_raw"] = pd.to_numeric(df["castness"], errors="coerce")

    # D: Penumbra continuity (softness, log-scaled)
    df["LF_penumbra_continuity_raw"] = np.log1p(
        pd.to_numeric(df["penumbra_softness"], errors="coerce").clip(lower=0)
    )

    # E: Light-induced depth ordering
    df["LF_depth_ordering_raw"] = pd.to_numeric(df["fg_bg_light_contrast"], errors="coerce")

    # F: Attention bias via light
    df["LF_attention_bias_raw"] = pd.to_numeric(df["attention_entropy"], errors="coerce")

    # ---- 2) Z-scored bucket metrics (global) ----
    df["LF_brightness_contrast_z"]  = zscore_global(df["LF_brightness_contrast_raw"])
    df["LF_illum_smoothness_z"]     = -zscore_global(df["LF_illum_smoothness_raw"])  # invert: smoother => higher
    df["LF_shadow_structure_z"]     = zscore_global(df["LF_shadow_structure_raw"])
    df["LF_penumbra_continuity_z"]  = zscore_global(df["LF_penumbra_continuity_raw"])
    df["LF_depth_ordering_z"]       = zscore_global(df["LF_depth_ordering_raw"])
    df["LF_attention_bias_z"]       = zscore_global(df["LF_attention_bias_raw"])

    # ---- 3) Light Organization Indices ----
    core_z = [
        "LF_brightness_contrast_z",
        "LF_illum_smoothness_z",
        "LF_shadow_structure_z",
        "LF_penumbra_continuity_z",
        "LF_depth_ordering_z",
    ]
    df["LightOrganizationIndex_core"] = df[core_z].mean(axis=1, skipna=True)

    all_z = core_z + ["LF_attention_bias_z"]
    df["LightOrganizationIndex_all"] = df[all_z].mean(axis=1, skipna=True)

    # ---- 4) Return compact LFV ----
    out_cols = [
        "split","image_path",
        "LF_brightness_contrast_raw","LF_illum_smoothness_raw","LF_shadow_structure_raw",
        "LF_penumbra_continuity_raw","LF_depth_ordering_raw","LF_attention_bias_raw",
        "LF_brightness_contrast_z","LF_illum_smoothness_z","LF_shadow_structure_z",
        "LF_penumbra_continuity_z","LF_depth_ordering_z","LF_attention_bias_z",
        "LightOrganizationIndex_core","LightOrganizationIndex_all"
    ]
    return df[out_cols].copy()

# ---------- 8.3 Run Step 8 ----------
df_light_base = build_light_base(df7=df7, df1=df1, df3=df3)

print("Missingness report (Step 8 inputs):")
print(light_base_missing_report(df_light_base))

df_light = build_light_feature_vector_minimal(df_light_base)

display(df_light.head())
print("\nIndex summary:")
display(df_light.groupby("split")[["LightOrganizationIndex_core","LightOrganizationIndex_all"]].describe())

# ---------- 8.4 Export ----------
out_csv = Path("artifacts/light_step8/light_feature_vector.csv")
out_csv.parent.mkdir(parents=True, exist_ok=True)
df_light.to_csv(out_csv, index=False)
print("\nSaved:", out_csv)

Missingness report (Step 8 inputs):
retinex_std             0
S_tv_mean               0
castness                0
penumbra_softness       0
fg_bg_light_contrast    0
attention_entropy       0
dtype: int64


,split,image_path,LF_brightness_contrast_raw,LF_illum_smoothness_raw,LF_shadow_structure_raw,LF_penumbra_continuity_raw,LF_depth_ordering_raw,LF_attention_bias_raw,LF_brightness_contrast_z,LF_illum_smoothness_z,LF_shadow_structure_z,LF_penumbra_continuity_z,LF_depth_ordering_z,LF_attention_bias_z,LightOrganizationIndex_core,LightOrganizationIndex_all
0,real,data\real\images\10pm_feeding_around_the_clock...,0.222949,0.002742,0.813970,5.230966,0.693374,12.842595,-0.049514,0.740972,-0.866999,-0.030925,-0.089884,0.767008,-0.059270,0.078443
1,real,data\real\images\11pm.webp,0.194081,0.003001,0.879319,4.917220,0.725034,12.256737,-0.934157,0.483577,-0.112252,-0.959613,0.260975,-0.354752,-0.252294,-0.269370
2,real,data\real\images\12_4_7_10.webp,0.214318,0.003563,0.948892,4.960868,0.542771,12.727888,-0.314024,-0.074924,0.691271,-0.830415,-1.758877,0.547375,-0.457394,-0.289932
3,real,data\real\images\around_the_clock.webp,0.245091,0.002652,0.962611,5.454100,0.710366,12.981635,0.629000,0.831294,0.849715,0.629550,0.098426,1.033232,0.607597,0.678536
4,real,data\real\images\around_the_clock_alizarin.webp,0.241964,0.002743,0.935668,5.188578,0.719899,13.055704,0.533158,0.740154,0.538538,-0.156395,0.204076,1.175054,0.371906,0.505764



Index summary:


LightOrganizationIndex_core                                          \
                                count      mean       std       min       25%   
split                                                                           
generated                        65.0  0.013895  0.500359 -0.978698 -0.353992   
real                             65.0 -0.013895  0.351527 -0.815402 -0.242488   

                                        LightOrganizationIndex_all            \
                50%       75%       max                      count      mean   
split                                                                          
generated  0.032444  0.298390  1.125823                       65.0 -0.083698   
real      -0.059270  0.236455  0.912287                       65.0  0.083698   

                                                                       
                std       min       25%       50%       75%       max  
split                                                                  
generated  0.416998 -0.865634 -0.395716 -0.082223  0.172405  0.857699  
real       0.406381 -0.668260 -0.180314  0.010801  0.298076  1.266034


Saved: artifacts\light_step8\light_feature_vector.csv
